In [26]:
#IMPORTAÇÃO BIBLIOTECAS
import pandas as pd
import numpy as np
import unidecode
import re

In [27]:
#IMPORTAÇÃO BIBLIOTECAS
import warnings
warnings.filterwarnings('ignore')

In [28]:
#LEITURA DO DATASET DE ACORDÃOS DO CARF, OBTIDO ATRAVÉS DO SCRAPER
acordaos = pd.read_csv("../Dataset CARF COFINS/DADOS ACORDAOS COFINS CARF 2016-2020.csv", sep="#")
acordaos.head

<bound method NDFrame.head of                                        NOME CONTRIBUINTE  \
0                   EMPRESA DE TRANSPORTES SAO LUIZ LTDA   
1        CITRATUS FRAGRANCIAS INDUSTRIA E COMERCIO LTDA.   
2      UNIMED DE FRANCA SOC COOPDE SERVICOS MED E HOS...   
3                           TELELISTAS ( REGIAO 1 ) LTDA   
4      TOV CORRETORA DE CAMBIO, TITULOS E VALORES MOB...   
...                                                  ...   
16082              RAYES E FAGUNDES ADVOGADOS ASSOCIADOS   
16083                       VOTORANTIM CIMENTOS N/NE S/A   
16084                       VOTORANTIM CIMENTOS N/NE S/A   
16085     COOPERATIVA AGROPECUARIA DE PEDRINHAS PAULISTA   
16086     COOPERATIVA AGROPECUARIA DE PEDRINHAS PAULISTA   

                         TIPO RECURSO DATA SESSAO  \
0                  RECURSO VOLUNTARIO  19/11/2009   
1                  RECURSO VOLUNTARIO  23/04/2014   
2                  RECURSO VOLUNTARIO  13/12/2016   
3                  RECURSO VOLUNTARIO

In [29]:
#FILTRANDO O DATASET DE ACORDÃOS PELOS ANOS DESEJADOS, PARA ELIMINAR DADOS EXTRAS ORIUNDOS DE FALHA NO FILTRO DO SITE DO CARF
print(len(acordaos))
anosAnalise = ["2016", "2017", "2018", "2019", "2020"]
acordaos = acordaos[acordaos["DATA SESSAO"].str.endswith(tuple(anosAnalise))]
print(len(acordaos))

16087
15841


In [30]:
#CRIA FUNÇÃO PARA OBTER REGEX DE UM TEXTO QUALQUER
def executarRegex(regex, texto, numGroup):
    txtSearch = re.search(regex, texto)
    if txtSearch != None:
        return txtSearch.group(numGroup)
    else:
        return None

In [31]:
#CRIA FUNÇÃO PARA EXTRAIR O PERÍODO DE APURAÇÃO A PARTIR DA COLUNA EMENTA
def extraiPeriodoApuracao(ementa):
    ementaSemAcentos = unidecode.unidecode(ementa).upper()
    periodoApuracao = None
    
    periodoApuracao = executarRegex("PERIODO DE APURACAO(:|.)? *[0-9]{2}/[0-9]{2}/[0-9]{4} *A *[0-9]{2}/[0-9]{2}/([0-9]{4})", ementaSemAcentos, 2)
    
    if periodoApuracao == None:
        periodoApuracao = executarRegex("DATA DO FATO GERADOR(:|.)? *[0-9]{2}/[0-9]{2}/([0-9]{4})", ementaSemAcentos, 2)
        
    if periodoApuracao == None:
        periodoApuracao = executarRegex("ANO(-| *)CALENDARIO(:|.)? *([0-9]{4}) *,? *A? *([0-9]{4})", ementaSemAcentos, 4)
        
    if periodoApuracao == None:
        periodoApuracao = executarRegex("ANO(-| *)CALENDARIO(:|.)? *([0-9]{4})", ementaSemAcentos, 3)
        
    if periodoApuracao == None:
        periodoApuracao = executarRegex("EXERCICIO(:|.)? *([0-9]{4}) *,? *A? *([0-9]{4})", ementaSemAcentos, 3)
        
    if periodoApuracao == None:
        periodoApuracao = executarRegex("EXERCICIO(:|.)? *([0-9]{4})", ementaSemAcentos, 2)
    
    return periodoApuracao

In [32]:
#EXTRAI O ANO DE APURAÇÃO A PARTIR DA COLUNA EMENTA
acordaos["ANO APURACAO"] = [extraiPeriodoApuracao(ementa) for ementa in acordaos["EMENTA"]]

#DESCONSIDERA AS LINHAS QUE NÃO POSSUEM ANO DE APURAÇÃO
print(len(acordaos))
acordaos.dropna(subset=["ANO APURACAO"], inplace=True)
print(len(acordaos))

15841
15834


In [33]:
#CRIA NOVA COLUNA NO DATASET DE ACÓRDÃOS PARA ARMAZENAR O ANO DO ACÓRDÃO A PARTIR DA DATA DA SESSÃO
acordaos["ANO ACORDAO"] = [executarRegex("[0-9]{2}/[0-9]{2}/([0-9]{4})", dataSessao, 1) for dataSessao in acordaos["DATA SESSAO"]]
acordaos["ANO ACORDAO"]

2        2016
3        2016
4        2016
5        2016
6        2016
         ... 
16082    2019
16083    2019
16084    2019
16085    2019
16086    2019
Name: ANO ACORDAO, Length: 15834, dtype: object

In [34]:
#FAZ TRATAMENTOS NA COLUNA EMENTA PARA FACILITAR OPERAÇÕES POSTERIORES
acordaos["EMENTA TRATADA"] = acordaos["EMENTA"].replace(r'\n',' ', regex=True) 
acordaos["EMENTA TRATADA"] = acordaos["EMENTA TRATADA"].str.replace("  ", " ")

In [35]:
#EXTRAI DA COLUNA EMENTA O ASSUNTO DO ACÓRDÃO, E ARMAZENA O RESULTADO EM NOVA COLUNA
def extraiAssunto(ementa):
    assunto = linhasEmenta = ementa.split("<br/>")[0]
    return assunto

acordaos["ASSUNTO"] = [extraiAssunto(ementa) for ementa in acordaos["EMENTA TRATADA"]]

In [36]:
#CRIA FUNÇÃO PARA FILTRAR APENAS OS ACÓRDÃOS COM ASSUNTO COFINS, A FIM DE CORRIGIR FALTA DE FILTRO ESPECÍFICO NO SITE DO CARF
regexAssuntoCOFINS = "cofins|contribuicao para o financiamento|financiamento da segurid"

def filtrarAcordaosCOFINS(acordaos):
    dadosFiltrados = []
    for i, linha in acordaos.iterrows():
        assunto = linha["ASSUNTO"]
        assunto = unidecode.unidecode(assunto).lower()
        if re.search(regexAssuntoCOFINS, assunto):
            dadosLinha = acordaos.loc[i].to_dict()
            dadosFiltrados.append(dadosLinha)
    
    listaColunas = acordaos.columns.values.tolist()
    return pd.DataFrame(dadosFiltrados, columns = listaColunas)

In [37]:
#FILTRA APENAS OS ACÓRDÃOS COM ASSUNTO COFINS
print(len(acordaos))
acordaos = filtrarAcordaosCOFINS(acordaos)
print(len(acordaos))

15834
9266


In [38]:
#CRIA FUNÇÃO PARA DIVIDIR AS EMENTAS, TENDO EM VISTA QUE UM ACÓRDÃO PODE POSSUIR VÁRIAS EMENTAS, E NO SITE DO CARF FICAM
#TODAS EM UM MESMO CAMPO
def dividirEmentas(acordaos):
    dadosEmentasDivididas = []
    for i, linha in acordaos.iterrows():
        ementas = linha["EMENTA TRATADA"]
        linhasEmentas = ementas.split("<br/>")
        
        anteriorUpper = False
        ementaAtual = None
        ementaAtualSalva = False
        tituloEmenta = None
        textoEmenta = None
        for le in linhasEmentas:
            if "ASSUNTO:" in le:
                anteriorUpper = False
                continue;
                
            if extraiPeriodoApuracao(le) != None:
                anteriorUpper = False
                continue;
            
            if le.isupper() and not anteriorUpper:
                ementaAtual = le
                anteriorUpper = True
                ementaAtualSalva = False
            elif le.isupper() and anteriorUpper:
                ementaAtual = ementaAtual + " " + le
                ementaAtualSalva = False
            elif not le.isupper() and ementaAtual != None:
                tituloEmenta = ementaAtual
                textoEmenta = le
                ementaAtual = ementaAtual + " " + le
                
                dadosLinha = acordaos.loc[i].to_dict()
                dadosLinha["EMENTA TRATADA"] = ementaAtual
                dadosLinha["TITULO EMENTA"] = tituloEmenta
                dadosLinha["TEXTO EMENTA"] = textoEmenta
                dadosEmentasDivididas.append(dadosLinha)
                
                anteriorUpper = False
                ementaAtualSalva = True
                ementaAtual = None
        
        if not ementaAtualSalva and ementaAtual != None:
            dadosLinha = acordaos.loc[i].to_dict()
            dadosLinha["EMENTA TRATADA"] = ementaAtual
            dadosLinha["TITULO EMENTA"] = tituloEmenta
            dadosLinha["TEXTO EMENTA"] = textoEmenta
            dadosEmentasDivididas.append(dadosLinha)
    
    listaColunas = acordaos.columns.values.tolist()
    listaColunas.append("TITULO EMENTA")
    listaColunas.append("TEXTO EMENTA")
    
    dfAcordaos = pd.DataFrame(dadosEmentasDivididas, columns = listaColunas)
    return dfAcordaos

In [39]:
#EXECUTA A FUNÇÃO DE DIVIDIR EMENTAS, DE FORMA QUE NOVAS LINHAS SERÃO ADICIONADAS AO DATASET PARA CADA EMENTA DIVIDIDA
acordaosED = dividirEmentas(acordaos)
acordaosED.head

<bound method NDFrame.head of                                        NOME CONTRIBUINTE  \
0                           TELELISTAS ( REGIAO 1 ) LTDA   
1                           TELELISTAS ( REGIAO 1 ) LTDA   
2      WYNY DO BRASIL INDUSTRIA E COMERCIO DE COUROS ...   
3               OPERADORA E AGENCIA DE VIAGENS TUR LTDA.   
4         GRAND BRASIL COMERCIO DE VEICULOS E PECAS LTDA   
...                                                  ...   
17879              RAYES E FAGUNDES ADVOGADOS ASSOCIADOS   
17880              RAYES E FAGUNDES ADVOGADOS ASSOCIADOS   
17881                       VOTORANTIM CIMENTOS N/NE S/A   
17882     COOPERATIVA AGROPECUARIA DE PEDRINHAS PAULISTA   
17883     COOPERATIVA AGROPECUARIA DE PEDRINHAS PAULISTA   

                 TIPO RECURSO DATA SESSAO  \
0          RECURSO VOLUNTARIO  13/12/2016   
1          RECURSO VOLUNTARIO  13/12/2016   
2          RECURSO VOLUNTARIO  13/12/2016   
3      EMBARGOS DE DECLARAÇÃO  23/11/2016   
4          RECURSO VOLUN

In [40]:
#LEITURA DO DATASET DE DADOS DO CNPJ, OBTIDO ATRAVÉS DO BANCO DE DADOS SQLITE
cnpjs = pd.read_csv("../Dataset CNPJ/DADOS CNPJ.csv", sep="#")
cnpjs.head

<bound method NDFrame.head of                                            razao_social  cnae_fiscal
0                                    BANCO DO BRASIL SA      6422100
1                     CENTRAIS ELETRICAS BRASILEIRAS SA      6462000
2                DISTRIBUIDORA BRASILIA DE VEICULOS S/A      4530703
3             CODIPE COMERCIAL DE PECAS E VEICULOS LTDA      4511101
4          ELO COMERCIO DE COMPONENTES AUTOMOTIVOS LTDA      4530701
...                                                 ...          ...
4244                         TOP VISION CALCADOS EIRELI      1531901
4245                      IRMAOS KUNST CONSTRUCOES LTDA      4120400
4246  VULCABRAS AZALEIA-RS,CALCADOS E ARTIGOS ESPORT...      1531901
4247                         SUPER MERCADO RISPOLI LTDA      4711302
4248              MECANICA COMERCIAL E IMPORTADORA LTDA      4511104

[4249 rows x 2 columns]>

In [41]:
#CRIA FUNÇÃO PARA PADRONIZAR NOMES DOS CONTRIBUINTES PARA POSTERIOR JUNÇÃO E FILTRO DE CNPJS
def preparaNomeParaJuncao(nome):
    if isinstance(nome, float):
        print(nome)
    nomeJuncao = nome.upper().replace(
        " EM RECUPERACAO JUDICIAL", ""
    ).replace(
        " FALIDO EM RECUPERACAO JUDICIAL", ""
    ).replace(
        "S.A.", "SA"
    ).replace(
        "S/A", "SA"
    ).replace(
        "&", "E"
    )
    
    return nomeJuncao

In [42]:
#PADRONIZA A COLUNA NOME CONTRIBUINTE DO DATASET DE ACORDAOS PARA POSTERIOR JUNÇÃO COM O DATASET DE CNPJ
acordaosED["NOME CONTRIBUINTE JUNCAO"] = [preparaNomeParaJuncao(nome) for nome in acordaosED["NOME CONTRIBUINTE"]]
#PADRONIZA A COLUNA RAZAO SOCIAL DO DATASET DE CNPJS PARA POSTERIOR JUNÇÃO COM O DATASET DE ACÓRDÃOS
cnpjs["razao_social_juncao"] = [preparaNomeParaJuncao(nome) for nome in cnpjs["razao_social"]]

In [43]:
#REALIZA A JUNÇÃO ENTRE OS DATASETS DE ACÓRDÃOS E CNPJS, USANDO COMO PARÂMETRO O NOME DO CONTRIBUINTE
acordaos_cnpj = pd.merge(acordaosED, cnpjs, left_on="NOME CONTRIBUINTE JUNCAO", right_on="razao_social_juncao")
acordaos_cnpj.head

<bound method NDFrame.head of                                        NOME CONTRIBUINTE  \
0               OPERADORA E AGENCIA DE VIAGENS TUR LTDA.   
1         GRAND BRASIL COMERCIO DE VEICULOS E PECAS LTDA   
2         GRAND BRASIL COMERCIO DE VEICULOS E PECAS LTDA   
3         GRAND BRASIL COMERCIO DE VEICULOS E PECAS LTDA   
4         GRAND BRASIL COMERCIO DE VEICULOS E PECAS LTDA   
...                                                  ...   
17810                  FIRENZE ACABAMENTOS EM COURO LTDA   
17811  MANTEP-MANUTENCAO PROJETOS E OBRAS INDUSTRIAIS...   
17812              RAYES E FAGUNDES ADVOGADOS ASSOCIADOS   
17813              RAYES E FAGUNDES ADVOGADOS ASSOCIADOS   
17814                       VOTORANTIM CIMENTOS N/NE S/A   

                 TIPO RECURSO DATA SESSAO  \
0      EMBARGOS DE DECLARAÇÃO  23/11/2016   
1          RECURSO VOLUNTARIO  13/12/2016   
2          RECURSO VOLUNTARIO  13/12/2016   
3          RECURSO VOLUNTARIO  13/12/2016   
4          RECURSO VOLUN

In [44]:
#CARREGA A TABELA DE CNAE E FAZ A JUNÇÃO COM O DATASET ACORDAOS_CNPJ
cnae = pd.read_csv("../Dataset CNPJ/tab_cnae.csv", sep="#")
acordaos_cnpj["cnae_fiscal"] = acordaos_cnpj["cnae_fiscal"].astype(int)
dadosJuntados = pd.merge(acordaos_cnpj, cnae, left_on="cnae_fiscal", right_on="cod_cnae")
dadosJuntados.head

<bound method NDFrame.head of                                     NOME CONTRIBUINTE            TIPO RECURSO  \
0            OPERADORA E AGENCIA DE VIAGENS TUR LTDA.  EMBARGOS DE DECLARAÇÃO   
1      CVC BRASIL OPERADORA E AGENCIA DE VIAGENS S.A.      RECURSO VOLUNTARIO   
2      CVC BRASIL OPERADORA E AGENCIA DE VIAGENS S.A.      RECURSO VOLUNTARIO   
3      CVC BRASIL OPERADORA E AGENCIA DE VIAGENS S.A.      RECURSO VOLUNTARIO   
4      CVC BRASIL OPERADORA E AGENCIA DE VIAGENS S.A.      RECURSO VOLUNTARIO   
...                                               ...                     ...   
17615        GRANITI BRAZIL INDUSTRIA E COMERCIO LTDA      RECURSO VOLUNTARIO   
17616        GRANITI BRAZIL INDUSTRIA E COMERCIO LTDA      RECURSO VOLUNTARIO   
17617        GRANITI BRAZIL INDUSTRIA E COMERCIO LTDA      RECURSO VOLUNTARIO   
17618     ORBIS TRUST SECURITIZADORA DE CREDITOS S.A.      RECURSO VOLUNTARIO   
17619               BS MATERIAIS PARA CONSTRUCAO LTDA      RECURSO VOLUNTARIO  

In [45]:
#REMOVE LINHAS CONSIDERADAS DUPLICADAS NO DATASET, COM BASE NO TITULO DA EMENTA, NOME CONTRIBUINTE E DATA SESSAO
#ISSO É NECESSÁRIO TENDO EM VISTA QUE A COFINS É UM TRIBUTO TRIMESTRAL, E É COMUM A ANÁLISE DE DIVERSOS TRIMESTRES DE UMA VEZ
#ENTRETANTO, CADA TRIMESTRE TERÁ PROCESSO PRÓPRIO, E, PORTANTO, ACÓRDÃO PRÓPRIO
#MAS APESAR DE TRIMESTRE DIFERENTES, É COMUM QUE O ASSUNTO SEJA O MESMO PARA TRIMESTRE PRÓXIMOS EM SEQUÊNCIA
#SENDO ASSIM, PARA EVITAR UM VÍCIO NA ANÁLISE, É NECESSÁRIO ELIMINAR EMENTAS DO MESMO CONTRIBUINTE EM SESSÕES NO MESMO DIA
#TENDO EM VISTA QUE GERALMENTE SE TRATAM DA MESMA ANÁLISE DIVIDIDA EM VÁRIOS PROCESSOS DE VÁRIOS TRIMESTRES
print(len(dadosJuntados))
dadosJuntados = dadosJuntados.drop_duplicates(subset=["TITULO EMENTA", "NOME CONTRIBUINTE", "DATA SESSAO"], keep="first", inplace=False)
print(len(dadosJuntados))

17620
6106


In [46]:
#ELIMINA COLUNAS DO DATASET QUE NÃO SERÃO MAIS NECESSÁRIAS, JÁ FORAM USADAS PARA JUNÇÃO E EXTRAÇÃO
dadosJuntados.pop("NOME CONTRIBUINTE JUNCAO")
dadosJuntados.pop("NOME CONTRIBUINTE")
dadosJuntados.pop("razao_social")
dadosJuntados.pop("razao_social_juncao")
dadosJuntados.pop("cnae_fiscal")
dadosJuntados.pop("DATA SESSAO")
dadosJuntados.pop("ASSUNTO")

0        Assunto: Contribuição para o Financiamento da ...
1        Assunto: Contribuição para o Financiamento da ...
3        Assunto: Contribuição para o Financiamento da ...
5        Assunto: Contribuição para o Financiamento da ...
6        Assunto: Contribuição para o Financiamento da ...
                               ...                        
17614    ASSUNTO: CONTRIBUIÇÃO PARA O FINANCIAMENTO DA ...
17615    ASSUNTO: CONTRIBUIÇÃO PARA O FINANCIAMENTO DA ...
17616    ASSUNTO: CONTRIBUIÇÃO PARA O FINANCIAMENTO DA ...
17617    ASSUNTO: CONTRIBUIÇÃO PARA O FINANCIAMENTO DA ...
17618    ASSUNTO: CONTRIBUIÇÃO PARA O FINANCIAMENTO DA ...
Name: ASSUNTO, Length: 6106, dtype: object

In [47]:
#SALVA O DATASET FINAL COMPLETO PARA POSTERIOR USO NA ANÁLISE DE DADOS E MACHINE LEARNING
dadosJuntados.to_csv("../Dataset COMPLETO/DATASET COMPLETO.csv", index=False, sep="#")